# Tool recommendation with Attention network 
## (Gated recurrent units Attention neural network with weighted cross-entropy loss)

In [19]:
import numpy as np
import json
import warnings
import operator

import tensorflow as tf
from keras import backend as K
import h5py

warnings.filterwarnings("ignore")


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        self.units = units
        super(BahdanauAttention, self).__init__(**kwargs)

    def get_config(self):
        return super().get_config().copy()

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


class ToolPredictionAttentionModel():
  
    def __init__(self, parameters):
        self.embedding_size = int(parameters["embedding_size"])
        self.gru_units = int(parameters["gru_units"])
        self.max_len = parameters["max_len"]
        self.dimensions = parameters["dimensions"]
        self.learning_rate = parameters["learning_rate"]
        self.class_weights = parameters["class_weights"]
        self.spatial_dropout = parameters["spatial_dropout"]
        self.recurrent_dropout = parameters["recurrent_dropout"]
        self.dropout = parameters["dropout"]

        
    def weighted_loss(self, class_weights):
        """
        Create a weighted loss function. Penalise the misclassification
        of classes more with the higher usage
        """
        weight_values = list(class_weights.values())

        def weighted_binary_crossentropy(y_true, y_pred):
            # add another dimension to compute dot product
            expanded_weights = tf.keras.backend.expand_dims(weight_values, axis=-1)
            return tf.keras.backend.dot(tf.keras.backend.binary_crossentropy(y_true, y_pred), expanded_weights)
        return weighted_binary_crossentropy

    def create_model(self):
        sequence_input = tf.keras.layers.Input(shape=(self.max_len,), dtype='int32')
        embedded_sequences = tf.keras.layers.Embedding(self.dimensions, self.embedding_size, input_length=self.max_len, mask_zero=True)(sequence_input)
        embedded_sequences = tf.keras.layers.SpatialDropout1D(self.spatial_dropout)(embedded_sequences)
        gru_output, h_forward, h_backward = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(self.gru_units,
                return_sequences=True,
                return_state=True,
                activation='elu',
                recurrent_dropout=self.recurrent_dropout
            ))(embedded_sequences)
        
        gru_hidden = tf.keras.layers.Concatenate()([h_forward, h_backward])
        
        '''gru_2 = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=True,
            activation='elu',
            recurrent_dropout=self.recurrent_dropout
        )'''
        
        #gru_output = gru_1(embedded_sequences)
        
        gru_output = tf.keras.layers.Dropout(self.dropout)(gru_output)
        
        #gru_output, gru_hidden = gru_2(gru_output)
        
        attention = BahdanauAttention(self.gru_units)
        context_vector, attention_weights = attention(gru_hidden, gru_output)
        dropout = tf.keras.layers.Dropout(self.dropout)(context_vector)
        output = tf.keras.layers.Dense(self.dimensions, activation='sigmoid')(dropout)
        model = tf.keras.Model(inputs=sequence_input, outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate),
            loss=self.weighted_loss(self.class_weights),
        )
        return model


def load_model(model_path):
    trained_model = h5py.File(model_path, 'r')
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    best_parameters = json.loads(trained_model.get('parameters').value)
    compatible_tools = json.loads(trained_model.get('compatible_tools').value)
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    new_model = ToolPredictionAttentionModel(best_parameters).create_model()
    new_model.load_weights(model_path)
    return dictionary, reverse_dictionary, best_parameters, compatible_tools, new_model


model_path = "data/tool_recommendation_attention_model.hdf5"
dictionary, reverse_dictionary, best_parameters, compatible_tools, new_model = load_model(model_path)

## Unpack trained model for prediction

In [20]:
print(reverse_dictionary)

{'1': 'fastq_quality_trimmer', '2': 'vcfallelicprimitives', '3': 'gatk2_variant_annotator', '4': 'Datamash', '5': 'proteomics_search_protein_prophet_1', '6': 'tophat', '7': 'cshl_uniq_tool', '8': 'deeptools_heatmapper', '9': 'cor2', '10': 'deeptools_bam_coverage', '11': 'samtool_filter2', '12': 'sam_merge2', '13': 'bwa_wrapper', '14': 'cuffdiff', '15': 'coords2clnt.py', '16': 'tp_replace_in_line', '17': 'fastq_to_fasta_python', '18': 'samtools_mpileup', '19': 'tp_cat', '20': 'hisat', '21': 'bedtools_genomecoveragebed_bedgraph', '22': 'snpSift_annotate', '23': 'fastq_to_tabular', '24': 'picard_FixMateInformation', '25': 'Add_a_column1', '26': 'picard_CollectInsertSizeMetrics', '27': 'modencode_peakcalling_macs2', '28': 'vcftools_merge', '29': 'Cut1', '30': 'tp_sort_header_tool', '31': 'rm_spurious_events.py', '32': 'bedtools_bedtobam', '33': 'ncbi_rpsblast_wrapper', '34': 'flexbar_split_RR_bcs', '35': 'regex_replace', '36': 'filter_bed_on_splice_junctions', '37': 'rseqc_inner_distance',

In [27]:
def compute_recommendations(tool_sequence, labels, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    # predict next tools for a test path
    prediction = new_model.predict(sample_reshaped, verbose=0)
    class_weights = best_parameters["class_weights"]
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    prediction = np.reshape(prediction, (prediction.shape[1],))
    #prediction = prediction * weight_val
    prediction = prediction / float(np.max(prediction))
    prediction_pos = np.argsort(prediction, axis=-1)
    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_pos = [item for item in topk_prediction_pos if item != 0]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Recommended tools for the tool sequence '%s' with their scores in decreasing order:" % tool_seq_name)
    print()
    for i in pred_tool_ids_sorted:
        print(i + "(" + str(pred_tool_ids_sorted[i]) + "%)")
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids:")
    print()
    for i in ids_tools:
        print(i + "(" + str(ids_tools[i]) + ")")

## Indices of tools

In [28]:
topk = 50 # set the maximum number of recommendations
tool_seq = "192" # give tools ids in a sequence and see the recommendations. To know all the tool ids, 
                     # please print the variable 'reverse_dictionary'
compute_recommendations(tool_seq, "", topk)

Current tool sequence: 

trimmomatic

Recommended tools for the tool sequence 'trimmomatic' with their scores in decreasing order:

tp_cut_tool(100%)
Cut1(41%)
Grep1(33%)
wig_to_bigWig(29%)
Add_a_column1(22%)
bedtools_intersectbed(20%)
gops_intersect_1(19%)
tp_replace_in_column(14%)
Grouping1(14%)
Remove beginning1(13%)
bg_uniq(13%)
cuffdiff(11%)
addValue(11%)
tp_sort_header_tool(11%)
Paste1(11%)
Filter1(10%)
Convert characters1(9%)
comp1(8%)
join1(7%)
sam_to_bam(7%)
sam_merge2(7%)
tp_easyjoin_tool(7%)
samtools_flagstat(7%)
Summary_Statistics1(6%)
tophat2(6%)
Extract_features1(6%)
tab2fasta(6%)
tp_replace_in_line(4%)
ProteinQuantifier(4%)
Show beginning1(4%)
seq_filter_by_id(4%)
samtools_sort(4%)
regex_replace(3%)
wc_gnu(3%)
cuffmerge(3%)
fastq_to_fasta_python(2%)
bamFilter(2%)
PeptideIndexer(2%)
methtools_dmr(2%)
samtools_rmdup(2%)
hgv_david(2%)
bowtie2(2%)
cat1(2%)
blastxml_to_top_descr(2%)
sort1(2%)
cufflinks(2%)
Extract genomic DNA 1(1%)
fastqc(1%)
bedtools_sortbed(1%)
IDMapper(1%)

## Recommended tools

In [23]:
print(new_model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 25, 82)       25420       input_3[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 25, 82)       0           embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 25, 464), (N 439872      spatial_dropout1d_2[0][0]        
____________________________________________________________________________________________

In [24]:
print(best_parameters)

{'embedding_size': 82, 'gru_units': 232, 'spatial_dropout': 0.15000000000000002, 'dropout': 0.25, 'recurrent_dropout': 0.0, 'learning_rate': 0.00836011725037937, 'batch_size': 32, 'clip_norm': 0.5, 'max_len': 25, 'dimensions': 310, 'class_weights': {'0': 0.0, '1': 2.282374674422305, '2': 5.913932166319434, '3': 0.09538827675467865, '4': 0.0, '5': 0.0869905767733068, '6': 0.0, '7': 0.07250647687303469, '8': 0.0, '9': 1.650222229444567, '10': 6.97617687621502, '11': 5.8328269641651795, '12': 5.6680003185450945, '13': 0.09530577070369572, '14': 3.698178191785774, '15': 0.6600210747001517, '16': 5.330374911417734, '17': 6.509065023535706, '18': 4.6549122778829055, '19': 5.26830770223046, '20': 0.0, '21': 0.0, '22': 3.988520976400871, '23': 1.9168309432747128, '24': 0.6018501587539669, '25': 5.888814776503156, '26': 2.451005098112319, '27': 0.0, '28': 2.3826786024540727, '29': 7.160087138220143, '30': 6.538477441630863, '31': 0.6600210747001517, '32': 4.382679657408116, '33': 1.581883693751